Юхимюк Роман

In [69]:
import gensim.models as gm
import pandas as pd
import string
import spacy
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
from sklearn.metrics import recall_score, accuracy_score, matthews_corrcoef, precision_score
from sklearn.metrics import classification_report, confusion_matrix

ПІДГОТОВКА

In [2]:
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
stemmer = SnowballStemmer(language='english')

In [3]:
#=================================================
# 1
def delete_non_literal(txt):
    ascii = (string.ascii_letters + " ")
    data1 = (c for c in txt if c in ascii)
    txt1 = "".join(data1)
    return ' '.join(txt1.split())


# 2
def tokenization(txt):
    return [str(tok) for tok in txt]


# 3
def delete_stopwords(tokenn):
    return [tok for tok in tokenn if not tok in spacy_stopwords]


# 4
def case_change(tokenn):
    return [tok.lower() for tok in tokenn]


# 5
def stemming(tokenn):
    return [stemmer.stem(tok) for tok in tokenn]


# 6
def lemmatization(txt):
    return [tok.lemma_ for tok in txt]


def all(txt):
    x = nlp(delete_non_literal(txt))
    a = tokenization(x)
    b = delete_stopwords(case_change(a))
    c = lemmatization(nlp(' '.join(b)))
    return stemming(c)
#===================================

Датасет текстів і міток(оцінка користувачів від 1 до 5):

In [6]:
df = pd.read_csv('APPLE_iPhone_SE.csv', encoding='utf8')
df = df[["Ratings", "Reviews"]].dropna()
df

,Ratings,Reviews
0,5,Great camera for pics and videos Battery life ...
1,5,Great device. Let me tell the Pros..1. Superb ...
2,5,"Who all loves older size i.e., 4.7 inch type s..."
3,5,This iPhone SE is the best phone ever you get....
4,5,This is my second iphone after iphone 4s. I’ve...
...,...,...
9708,5,Absolutely brilliantREAD MORE
9709,5,"Superb phone. This is my 4th iPhone, I feel SE..."
9710,5,very niceREAD MORE
9711,5,Loving it as of now. Good Product .READ MORE


In [43]:
df1 = df["Ratings"]
df2 = df["Reviews"]

Підготовка даних:

In [10]:
arr = []
for i in df2:
    lst = all(i)
    arr.append(lst)

In [12]:
arr1 = [' '.join(i) for i in arr]

Модель енкодера
Завантаження:

In [44]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

Перетворення текстів на вектори:

In [14]:
message_embeddings = embed(arr1)

Розбиття датасету:

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(np.array(message_embeddings),np.array(df1), test_size=0.2)

Модель класифікації на основі логістичної регресії:

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, Y_train)

Класифікаця тестового набору

In [25]:
pre = clf.predict(X_test)

Метрика повноти:

In [63]:
rs_ = recall_score(Y_test, pre,  average='weighted')
rs_

0.6891405043746783

Метрика точності:

In [64]:
as_ = accuracy_score(Y_test, pre)
as_

0.6891405043746783

Коефіціент кореляції Метьюза:

In [65]:
mc_ = matthews_corrcoef(Y_test, pre)
mc_

0.20691364935858622

Точність:

In [66]:
ps_ = precision_score(pre, Y_test, average='micro')

Матриця похибок:

In [67]:
confusion_matrix(Y_test, pre)

array([[  29,    0,    7,    7,   58],
       [  12,    1,    1,    5,   25],
       [   9,    1,    5,    9,   80],
       [   7,    1,    2,   20,  349],
       [   5,    0,    8,   18, 1284]])

Запис в файли:

In [61]:
import csv

with open('output.csv', 'w', encoding='UTF8', newline='') as fw:
    writer = csv.writer(fw)
    writer.writerow(['id_text', 'class'])
    for i, val in enumerate(pre):
        writer.writerow([i, val])

In [68]:
with open('metrics.txt', 'w', encoding='utf-8') as fw:
    fw.write("Accuracy = " + str(as_) + "\nPrecision = " + str(ps_) + 
             "\nRecall = " + str(rs_) + "\nMatthews Correlation Coefficient = " + str(mc_))